In [1]:
#TODO

"""
1. predict the unlabeled data (Naive Bayes, SVM)
2. evaluate the accuracy,f1,ruc,etc?? (check with the doc later)
3. Topic Modeling
4. wordcloud(maybe,if applicable)
5. visualization(maybe, if applicable)
6. think about some other fun questions about this data set

"""

'\n1. predict the unlabeled data (Naive Bayes, SVM)\n2. evaluate the accuracy,f1,ruc,etc?? (check with the doc later)\n3. Topic Modeling\n4. wordcloud(maybe,if applicable)\n5. visualization(maybe, if applicable)\n6. think about some other fun questions about this data set\n\n'

In [2]:
import pandas as pd


#settings
pd.options.display.max_rows=10

In [3]:
mapping = pd.read_csv('data/Interview_Mapping.csv')
mapping

,Judgements,Area.of.Law
0,LNIND_1988_CAL_114,To be Tested
1,LNIND_1956_CAL_163,To be Tested
2,LNIND_1976_CAL_277,To be Tested
3,LNIND_1980_CAL_52,To be Tested
4,LNIND_1955_CAL_124,To be Tested
...,...,...
994,LNIND_1993_DEL_112,Criminal Laws
995,LNIND_1988_CAL_83,Service Law
996,LNIND_1993_DEL_16,Criminal Laws
997,LNIND_1957_CAL_46,Succession Laws


In [27]:
# create labels
unlabeled = []
labeled = []
labels = []

for index,row in mapping.iterrows():
    if row['Area.of.Law'] == 'To be Tested':
        unlabeled.append(row['Judgements'])
    else: 
        labeled.append(row['Judgements'])
        labels.append(row['Area.of.Law'])
        
# how much unique area of law        
len(set(labels))

41

In [33]:
# load files
import os
count=0
unlabeled_text=[]
labeled_text=[]

for name in unlabeled:
    path = os.path.join('data/',name+'.txt')
    with open(path,'r',errors = 'ignore') as f:
        unlabeled_text.append(f.read())
for name in labeled:
    path = os.path.join('data/',name+'.txt')
    with open(path,'r',errors = 'ignore') as f:
        labeled_text.append(f.read())


"â€“ <KYDishonestly receiving stolen property Parties</KY>\nBharadwaj Singh Versus State Of West Bengal, on the complaint of, Harimati Dasi\nHigh Court of Judicature at Calcutta\nJudges THE HONOURABLE CHIEF JUSTICE MR. HARRIES\nCriminal Revn No. 300 of 1951DOJ 08.05.1951\nADVOCATES APPEARED:For the Appearing Parties : Kishore Mukharji,M.N.Banerji,N.C.Talukdar,Advocates.\nJudgment\nHARRIES, C. J.\n<FC>(1) THIS is a petition for revision of orders of the Courts below convicting the petitioner of an offence under   <PV>Section 411</PV>, <LG>I. P. C.</LG> , and sentencing him to pay a fine of Rs. 200/- and in default of payment of the said fine, to undergo rigorous imprisonment for three months.</FC>\n(2) THE charge against the petitioner was that he was in possession of a cow knowing the same to be stolen or having reasonable grounds for believing that the same was stolen. A learned Magistrate found him guilty and sentenced him as I have indicated. On appeal the conviction and sentence we

In [11]:
# Text pre-processing
import nltk
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
punct = set(string.punctuation)

#remove \n
#remove punctuation
#remove stopwords
#remove links
#normaliza text 
#tokenize
#lemmatization


TypeError: join() takes exactly one argument (3 given)